In [1]:
#importowanie bibliotek
import cv2
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

In [2]:
dataset = datasets.ImageFolder(
    'ca_dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [3]:
#podzielenie danych na trenowane i testowane
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

# test_percent = 0.1
# num_test = int(test_percent * len(dataset))
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

In [5]:
#zdefiniowanie sieci neuronowej resnet18
model = models.resnet18(pretrained=True)

In [ ]:
#zdefiniowanie sieci neuronowej alexnet
# model = models.alexnet(pretrained=True)


In [ ]:
# model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [6]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

In [7]:
NUM_EPOCHS = 100
BEST_MODEL_PATH = 'collision_model_sala230_100.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.800000
1: 1.000000
2: 0.900000
3: 0.960000
4: 0.940000
5: 0.940000
6: 0.980000
7: 0.920000
8: 1.000000
9: 0.980000
10: 0.940000
11: 0.960000
12: 0.960000
13: 0.980000
14: 0.960000
15: 0.880000
16: 0.980000
17: 0.980000
18: 0.900000
19: 1.000000
20: 0.980000
21: 0.960000
22: 0.960000
23: 1.000000
24: 1.000000
25: 1.000000
26: 1.000000
27: 1.000000
28: 0.960000
29: 0.980000
30: 1.000000
31: 0.960000
32: 0.960000
33: 1.000000
34: 1.000000
35: 1.000000
36: 1.000000
37: 0.980000
38: 1.000000
39: 1.000000
40: 0.980000
41: 0.960000
42: 1.000000
43: 1.000000
44: 1.000000
45: 1.000000
46: 1.000000
47: 1.000000
48: 1.000000
49: 0.980000
50: 1.000000
51: 1.000000
52: 0.920000
53: 0.960000
54: 1.000000
55: 0.960000
56: 0.960000
57: 0.960000
58: 0.960000
59: 0.960000
60: 1.000000
61: 0.960000
62: 0.940000
63: 0.980000
64: 1.000000
65: 1.000000
66: 0.860000
67: 1.000000
68: 0.960000
69: 1.000000
70: 1.000000
71: 1.000000
72: 1.000000
73: 1.000000
74: 1.000000
75: 0.960000
76: 1.000000
77: 1.000